# MARIDA Dataset exploration

#### Need to set up a folder for unified visualisation and qgis qml visualisation because the quality is the true definition of terrible.

In [1]:
from glob import glob

all_tifs = glob("datasets/MARIDA_sentinel-2/patches/**/*.tif", recursive=True)
print(len(all_tifs))

4143


In [7]:
import shutil
import re

# Compile a regex pattern to match "C_{number}.tif"
pattern = re.compile(r"C_\d+\_conf.tif$")

# Filter files using the regex pattern
filtered_files = [file for file in all_tifs if pattern.search(file)]

for file in filtered_files:
    shutil.copy(file, "datasets/MARIDA_sentinel-2/only_confs/")

In [8]:
import os

print(len(os.listdir("datasets/MARIDA_sentinel-2/only_confs/")))

1099
